## **`Paso 4`: Exploración de datos**

### **1. Exploración inicial del archivo `Unemployment.csv`**

#### **Paso 1 - Acceder a Azure Synapse Studio**

1. **Abrir Azure Synapse Studio**: 
    - Regresa al Azure Portal y busca `All resources`.
    - Selecciona tu instancia de **Azure Synapse Workspace**.
    - Haz clic en `Abrir Synapse Studio` para acceder a la interfaz de usuario de Synapse Studio.

[![p1193.png](https://i.postimg.cc/VLZz2B6b/p1193.png)](https://postimg.cc/KRLXMTtZ)
[![p1185.png](https://i.postimg.cc/tTYnPCYs/p1185.png)](https://postimg.cc/tZGgGbdb)

#### **Paso 2 - Navegar a la sección "Develop"**

1. En el panel izquierdo de Synapse Studio, selecciona la pestaña **Develop**.
2. Dentro de la sección **Develop**, haz clic en el botón `+ New` y selecciona `SQL Script`. Esto abrirá un editor de SQL donde podrás escribir tus consultas.
3. Renombremos el SQL Script como **1_Data_Exploration**.
4. En el lado derecho, puedes ver que está conectado a **Built-in**. Este es el **Serverless SQL pool** que obtenemos por defecto. Si ves la base de datos en el lado derecho, verás algo llamado **master**. Esta es la base de datos predeterminada que obtenemos de **Serverless SQL pool**. Cuando creo un nuevo script SQL, por defecto se conecta al **Serverless SQL pool**

[![p1195.png](https://i.postimg.cc/t4rD3JBb/p1195.png)](https://postimg.cc/yDZFB1qp)
[![p1196.png](https://i.postimg.cc/QxvG1cqQ/p1196.png)](https://postimg.cc/ZCPMhB2n)

#### **Paso 3 - Obtener la ruta del archivo**

1. **Seleccionar el archivo**:
    - Desde la sección **Data** dentro del contenedor `raw`, haz clic sobre el archivo `Unemployment.csv`.
    - En el panel de detalles, selecciona la opción `Properties`.

2. **Propiedades del archivo**:
    - Nos muestra las rutas `URL` y `ABFSS`.

[![p1197.png](https://i.postimg.cc/h4sMkmZ9/p1197.png)](https://postimg.cc/14gp68Qt)
[![p1198.png](https://i.postimg.cc/MKGtqGgg/p1198.png)](https://postimg.cc/9zvyBcmB)

#### **Paso 4 - Exploración del archivo utilizando `OPENROWSET()`**

Vamos a analizar el archivo utilizando la ruta **HTTPS**:

[![p1199.png](https://i.postimg.cc/ydTVsj2h/p1199.png)](https://postimg.cc/2bVpxQk3)

Vamos a analizar el archivo utilizando la ruta **ABFSS**:

[![p1201.png](https://i.postimg.cc/FzWNJRr7/p1201.png)](https://postimg.cc/PvwBgd8j)

Para obtener la fila de encabezado, vamos a mencionar el parámetro **HEADER_ROW** y necesitamos decir que es verdadero, indicar **TRUE**. En caso de que estés usando **PARSER_VERSION = 1.0**, no podras usar la función de **HEADER_ROW**. Por otro lado, Microsoft recomienda usar **PARSER_VERSION = 2.0** porque es mucho más eficiente. 

[![p1202.png](https://i.postimg.cc/G3qcfMFd/p1202.png)](https://postimg.cc/s1GdQ4HN)

En lugar de obtener todas las columnas, también podemos seleccionar solo columnas específicas. En lugar de usar `*` puedo definir directamente cualquiera de las columnas que quiero. Intentemos consultar el campo `Line Number` y `Month`. Dado que tengo espacios en el nombre del campo es preferible usar corchetes. 

[![p1203.png](https://i.postimg.cc/VvLz6Mcx/p1203.png)](https://postimg.cc/yg5MpDxj)

Luego, también podemos definir esto usando `alias`.

[![p1204.png](https://i.postimg.cc/GhtcnkKY/p1204.png)](https://postimg.cc/ZCkkrv1K)

Podemos usar la cláusula **WITH** para definir un schema externo, por ahora solo vamos a consultar dos campos: `Line Number` y `Month`. El primero lo definimos como tipo **INT** y el segundo como tipo **VARCHAR**, así que vamos a definir estos externamente.

[![p1205.png](https://i.postimg.cc/m2nTLtpn/p1205.png)](https://postimg.cc/0b7TZ2sG)

El uso de la opción **FIRSTROW** es en caso de que no estés usando **HEADER_ROW**. El uso de la función **FIRSTROW** es indicarle a la función **OPENROWSET()** que mi primera fila comenzará desde el número de línea 2.

[![p1206.png](https://i.postimg.cc/qvC4z49v/p1206.png)](https://postimg.cc/NKYWnqs3)

También tenemos una opción más llamada **FIELDTERMINATOR** al cual le indicamos el simbolo que se utiliza en el archivo para  separar las columnas. En nuestro caso, cada campo está separado por una coma. 

[![p1207.png](https://i.postimg.cc/HLDHMFg4/p1207.png)](https://postimg.cc/QFbRG4VH)

#### **Paso 5 - Arreglar el `Collation Warning`**

1. Crear un nuevo script SQL, solo debes hacer clic en los tres puntos y luego hacer clic en `New Script SQL`. Le daremos el nombre de **2_Fixing_collation_warning**

[![p1208.png](https://i.postimg.cc/Gmr61Dkr/p1208.png)](https://postimg.cc/dLH52LHS)

2. Ahora, si haces clic en `Messages`, verás una advertencia que indica un posible error de conversión al leer las columnas VARCHAR desde un texto codificado en UTF-8. 
3. Esto significa que obtuvimos estos datos de un archivo CSV que está codificado en UTF-8. Esta es una codificación común que veremos también en escenarios reales cuando tomemos los datos de cualquier archivo CSV. Para solucionar esto, nos sugiere dos posibles soluciones: 
    - Cambiar la collation de la base de datos a una que sea compatible con UTF8 
    - Especificar explícitamente el schema de la columna en la cláusula **WITH** y asignar la `UTF-8 collation` para cada columna **VARCHAR**. Actualmente, enfrentaremos este error con las columnas **VARCHAR**.

[![p1209.png](https://i.postimg.cc/s2g8MR6N/p1209.png)](https://postimg.cc/21gTtgfQ)

4. **Para corregirlo, primero vamos a crear una base de datos**. Actualmente, estamos usando la base de datos **master**, así que vamos a crear una nueva base de datos, ya que no se recomienda alterar la base de datos **master** si queremos cambiar la `collation` para toda la base de datos. La base de datos se llamará `ProjectDB`.
5. En la sección `Data` en la pestaña `Workspace` se mostrarán todos los metadatos relacionados con nuestro workspace en Synapse. Así que si estamos creando una base de datos, aparecerá en esta pestaña bajo `SQL Database`. Para esto es necesario publicar todo (`Publish all`). Necesitamos publicar para que todos los cambios se sincronicen, y luego actualizar la página. 

[![p1210.png](https://i.postimg.cc/Z5c2YqcN/p1210.png)](https://postimg.cc/VJJDDm2s)
[![p1211.png](https://i.postimg.cc/nhHSWJJh/p1211.png)](https://postimg.cc/2VKxqgys)
[![p1212.png](https://i.postimg.cc/1zXC9r4x/p1212.png)](https://postimg.cc/p9NYC8Dk)
[![p1213.png](https://i.postimg.cc/L5tvyyQP/p1213.png)](https://postimg.cc/5QtwymV9)

6. Ahora abordemos nuestros problemas corrigiendo la `collation`. Usamos la base de datos **ProjectDB**. Para corregir las advertencias de `collation`, se nos dice que podemos usar dos métodos: 

- Uno es definir explícitamente la `collation` para las columnas 
- El otro es definir la `collation` para toda la base de datos

Vamos a ver ambos enfoques, así que empecemos con el primero, que es definir explícitamente la `collation` para cada columna. Necesitamos definir `collation` en la cláusula **WITH**. Ahora, para definir `collation`, después de definir **VARCHAR** se debe ingresar **COLLATE**: 
```sql
-- USAR EXPLICITAMENTE COLLATION SOBRE LAS COLUMNAS

SELECT [data].[Line Number],[data].[Month]
FROM 
    OPENROWSET(
        BULK 'abfss://raw@adlsproyectos.dfs.core.windows.net/Unemployment.csv',
        FORMAT = 'CSV',
        HEADER_ROW = TRUE,
        PARSER_VERSION = '2.0'
    ) 
    WITH (
        [Line Number] VARCHAR(10) COLLATE Latin1_General_100_CI_AS_KS_SC_UTF8,
        [Month] VARCHAR(10) COLLATE Latin1_General_100_CI_AS_KS_SC_UTF8
    ) AS [data]
```
Ahora puedes ver que este es el tipo de `collation` con el que definimos las columnas. Vamos a ejecutarlo y vemos que lo hizo correctamente. Volvamos y hagamos clic en `Messages`. Ahora no estamos viendo ninguna advertencia de posibles problemas de `collation`. Solo nos dice que nuestra declaración se ejecutó y cuántas filas fueron afectadas. Esto solo es una confirmación de que nuestra consulta se ha ejecutado correctamente. Así es como podemos manejar los mensajes de `collation`, manejándolos explícitamente. 

[![p1214.png](https://i.postimg.cc/DfxSGQwp/p1214.png)](https://postimg.cc/qgCBV3n2)
[![p1215.png](https://i.postimg.cc/Kv4RDKKG/p1215.png)](https://postimg.cc/BX9Sqnck)

7. <mark>**Ahora veamos cómo manejar esto a nivel de toda la base de datos**</mark>. Para eso, necesitamos hacer un **ALTER DATABASE**. 

|La razón por la que creamos una base de datos es que si deseas cambiar la `collation` para toda la base de datos, no puedes hacerlo en la base de datos **master**|
|---| 

Así que vamos a cambiar la `collation` y vamos a alterar la base de datos para habilitar la `collation`. 
```sql
-- CAMBIAR COLLATION PARA TODA LA BASE DE DATOS

ALTER DATABASE ProjectDB
COLLATE Latin1_General_100_CI_AS_KS_SC_UTF8
```

[![p1216.png](https://i.postimg.cc/vTCMzZjT/p1216.png)](https://postimg.cc/sGPtfs3C)

Aquí no hemos definido explícitamente ninguna `collation` para las columnas:

[![p1217.png](https://i.postimg.cc/NGngYdMt/p1217.png)](https://postimg.cc/6TdsfLnb)
[![p1218.png](https://i.postimg.cc/3JD3w24v/p1218.png)](https://postimg.cc/8jD2KJbp)

#### **Paso 6 - Crear una `EXTERNAL TABLE` utilizando la sintáxis `CETAS`**

1. Crear un nuevo script SQL, solo debes hacer clic en los tres puntos y luego hacer clic en `New Script SQL`. Le daremos el nombre de **3_Creating_External_table**
2. Debemos mover los datos del contenedor `raw` al contenedor `refined`. Tenemos el dataset `Unemployment.csv`, que está en el contenedor `raw` y queremos llevarlo a un formato `parquet` en el contenedor `refined`.
3. Vamos a crear una **EXTERNAL TABLE** utilizando el resultado de una consulta **SELECT** (`CETAS`) donde no vamos a utilizar las columnas `InsertSource`, `Data Version`, `Access Permission` y `Update Frequency`
4. Esta **EXTERNAL TABLE** se creará en el contenedor `refined`
5. Para crear una **EXTERNAL TABLE** necesitamos crear **EXTERNAL FILE FORMAT** que indicará el formato `parquet` y **EXTERNAL DATA SOURCE** que hará referencia al contenedor `refined`

[![p1219.png](https://i.postimg.cc/NjBtvRkT/p1219.png)](https://postimg.cc/ykQtFg5Y)
[![p1220.png](https://i.postimg.cc/cLhSx6Fz/p1220.png)](https://postimg.cc/k22ZwM3N)

Veamos qué hicimos exactamente: 

- Escribimos una instrucción **SELECT** que solo traerá las filas específicas que necesitamos.
- Luego estamos creando una **EXTERNAL TABLE** usando `DATA_SOURCE` como `refined_path` que hace referencia al contenedor `refined`, `LOCATION` como `ExtNew` y FILE_FORMAT como `parquet_format`, donde definimos el formato de archivo como el formato `parquet`.
- Al escribir esto, se creará una nueva carpeta llamada `ExtNew`, dentro del contenedor `refined`, y dentro de esa carpeta se almacenará la información. 

En resumen:

Las opciones necesarias para crear la **EXTERNAL TABLE** son: 
- `DATA_SOURCE`: guarda la ruta del almacenamiento (en este caso, el contenedor `refined`)
- `LOCATION`: ruta que define en qué carpeta quieres almacenar los datos
- `FILE_FORMAT` que hace referencia al **EXTERNAL FILE FORMAT**, que define el formato en el que quieres almacenar la **EXTERNAL TABLE** en el almacenamiento de datos. 

[![p1221.png](https://i.postimg.cc/DzY3GHkd/p1221.png)](https://postimg.cc/KKLWS9YR)
[![p1222.png](https://i.postimg.cc/yNNzkB58/p1222.png)](https://postimg.cc/nXNWW8py)
[![p1223.png](https://i.postimg.cc/PxpsQydG/p1223.png)](https://postimg.cc/sQrNVpBK)

Si vamos a la sección `Data` y luego a la pestaña `Workspace` y desplegamos `External tables`, luego actualizamos y veremos que se muestra la **EXTERNAL TABLE** `dbo.Ext_Table`. Si actualizamos también las `External data sources`, podemos ver `refined_path`, y si actualizamos también los `External file formats`, veremos que aparece `parquet_format`. 

[![p1224.png](https://i.postimg.cc/xTWkvpYQ/p1224.png)](https://postimg.cc/Yhzqk3hD)

#### **Paso 7 - Verificar resultado**

1. Verificamos que se haya creado un archivo `parquet` en el directorio `ExtNew` dentro del contenedor `refined`.

[![p1225.png](https://i.postimg.cc/59gTQDBH/p1225.png)](https://postimg.cc/NLypWJPY)
[![p1226.png](https://i.postimg.cc/CLKtGJ7c/p1226.png)](https://postimg.cc/PNB2hWBD)